<a href="https://colab.research.google.com/github/arseniykotov02112010-dev/Pushkin/blob/main/Stih.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch

# Проверяем, доступен ли GPU (должно быть True)
print("GPU доступен:", torch.cuda.is_available())

# Если доступен, выводим его название
if torch.cuda.is_available():
    print("Название GPU:", torch.cuda.get_device_name(0))
    print("Объем памяти GPU:", torch.cuda.get_device_properties(0).total_memory / 1024**3, "ГБ")

GPU доступен: True
Название GPU: Tesla T4
Объем памяти GPU: 14.74127197265625 ГБ


In [3]:
!pip install -q transformers accelerate peft bitsandbytes
!pip install -q torch torchvision torchaudio
!pip install -U bitsandbytes
!pip uninstall -y bitsandbytes
!pip install -q bitsandbytes>=0.41.0
!pip install -q transformers>=4.35.0
!pip install -q accelerate>=0.24.0
!pip install -q peft>=0.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.9 MB/s eta 0:00:00
Found existing installation: bitsandbytes 0.49.0
Uninstalling bitsandbytes-0.49.0:
  Successfully uninstalled bitsandbytes-0.49.0


In [12]:
#СОЗДАНИЕ ДАТАСЕТА
poems = [
    """Бог поглядел с тоскою на восток,
налил в стакан сто грамм холодной водки
и опрокинул солнечный желток
на серый круг вечерней сковородки.""",
    """Задумайтесь, пока метет пурга
и отменили питерские рейсы -
о чем молчат альпийские луга,
когда на них ступают эдельвейсы?""",
    """Поскольку я писал без перерыва
И возносил приветствия луне,
Я думаю, что это справедливо,
что весь рассвет достался только мне…""",
    """Я кашлянул в звенящей тишине,
И от шального эха стало жутко…
Расскажет ли утятам обо мне
под утро мной испуганная утка?""",
    """Мне кажется, что я уже не рад
Тому, что прихватил с собою «Никон»…
А вдруг не влезет в фотоаппарат
вот этот остров в зарослях черники?""",
    """Когда прибой слизнул мои часы,
у времени сменились очертанья,
и на мои пшеничные усы
упала тень чужого мирозданья…""",
    """Через каких-то несколько минут
На этом поле станет очень чисто,
и мыши Геростратом назовут
в своих мышиных мифах тракториста.""",
    """Четвертый день октябрь, не спеша,
несет дожди на ветхом коромысле.
Промокло все: и куртка, и душа...
Откуда же взялись сухие мысли?""",
    """Я счастлив был почти четыре дня:
читал стихи, смеялся анекдотам,
покуда не окликнула меня
в трамвае нелюбимая работа…""",
    """Мне надоело жить и не тужить.
Хочу тужить, стихи свои листая.
Никто мне не желает подарить
банановую хижину в Китае?""",
    """Ужель светильник разума угас
И все поэты протянули ноги?
Ну, почему, поднявшись на Парнас,
я никого не встретил по дороге?""",
    """Весь мир идет к достойному концу,
Но всем назло останемся в живых мы.
Вы знаете, а вам стихи - к лицу...
Но для чего на шляпке эти рифмы?...""",
    """Иссякло вдохновение. Беда…
А я еще над Созиным смеялся…
Любимая, когда ты шла сюда,
тебе случайно ямб не попадался?""",
    """Пусть у меня другие интересы,
Но разузнать я буду очень рад –
Кого мне рок определил в Дантесы?
Какой Сальери мне готовит яд?""",
    """Зачем я прочитал про «кубок Феба»!?
Оставив тело в северной глуши,
Моя душа опять умчалась в небо.
Теперь сиди, как дурень, без души.""",
    """Вы свой приход на завтра отодвиньте
И поживите от меня вдали.
Я заблудился в книжном лабиринте...
И не хочу, чтобы меня нашли…""",
    """Когда в тридцатом умер Маяковский,
И все рыдали на похоронах,
Почти полдня по улицам московским,
Глумясь, носилось облако в штанах…""",
    """Я весь погряз в сомнениях своих.
Я знать хочу до умопомраченья:
Мир существует только для двоих...
К чему же остальное населенье?""",
    """Как наша жизнь устроена хитро…
Я знаю точно (бросьте кривотолки!):
Когда меня ударил бес в ребро,
Мой ангел был в банальной самоволке.""",
    """Ничто не исчезает в этом мире,
И лишь семья – загадка бытия.
Когда нас стало четверо в квартире,
Куда-то делись двое - ты и я…""",
    """Я знаю, что в последний день весны,
Вернее – в ночь, по нашему проулку,
Когда жильцы в кроватях видят сны,
дома друг друга водят на прогулку.""",
    """Пусть за глаза «придурком» называют,
Признаюсь перед всем Череповцом,
что мне встречались пьяные трамваи
и рельсы с человеческим лицом.""",
    """Я к водке не испытывал влеченья,
Но все же выпил посошок с сосной,
Поскольку знал: нуждается в леченьи
природа, заболевшая весной.""",
    """Коня, коня! Полцарства за коня!
Я полечу из мест, где вы живете,
поскольку точно знаю: для меня
вчера расцвет багульник на болоте…""",
    """Я как-то вдруг случайно осознал
сквозь телевизионную сутулость,
что стал вдаваться в телесериал...
Не слишком ли тоска подзатянулась?""",
    """Идут года, и ты уже не ждешь
Ни радости от жизни, ни кручины.
По радио сказали: будет дождь...
О, Господи, не все ли мне едино!""",
    """Я спас себя. Ну, а других спасти
Уже не смог по подлости закону…
Простите, я могу приобрести
сервиз кофейный на одну персону?""",
    """Живешь, шутя, и даже спишь, шутя…
А смерть придет – шутя, сойдешь в могилу…
Во сне ты улыбалась, как дитя...
И с кем ты мне сегодня изменила?""",
    """Я вляпался в собачий непокой,
Разбив апофеоз собачьей драки.
Ну, неужели я и впрямь такой,
каким меня облаяли собаки?""",
    """Вечерний лес усталостью прошит.
Иду и размышляю сквозь усталость:
Кто целый час смеркал окрестный вид?
Что именно вокруг меня смеркалось?""",
    """Прогонишь из костей остатки сна,
Шагнешь во тьму и выдохнешь: «О, Боже!
Ну, до чего вечерняя луна
на девочку невинную похожа!».""",
    """Вот и еще один месяц
тихо подходит к концу.
В поисках детской смеси
брожу по Череповцу...""",
    """Наконец-то я решился
на поступок героический:
встал под утро и побрился
бритвой электрической.""",
    """Я подумал и решил,
съев в кафе тарелку супа,
что работать за гроши -
это более, чем глупо...""",
    """Онемевшими руками
я вцепился в небосвод.
Боже мой, как с дураками
в этой жизни мне везет!""",
    """Детективное агенство
“Широглазов и семья”
ищет в мире совершенство
и счастливые края!""",
    """Я боюсь сам себя за припадки тоски,
за недобрую страсть к оружью
и за мысли, что рвут на куски виски,
потому что хотят наружу...""",
    """Я задумался. И пока
мои мысли рвались наружу,
я нечаянно обнаружил
в небе перистые облака...""",
    """Вы простите меня, дурака,
но я вывел закон такой:
покойник - от слова “пока”,
а не от слова “покой”...""",
    """Все. Конец. Хочу в тюрьму.
Хватит извращаться!
Вы не знаете - к кому
это обращаться?""",
    """Фонарь на краю света...
Я его зажигаю каждый вечер
и каждое утро гашу.
Больше некому...""",
    """Постучал - и мне открыли,
мне открыли - я вошел,
я вошел - и в этом мире
сразу стало хорошо...""",
    """-Эй, кукшка, сколько лет
жить мне причитается?
Замолчала птица. Нет!
Это не считается!""",
    """За этот год, что по-своему свят
в моем переменчивом мире,
я глупых книг прочитал 50,
а умных - только 4.""",
    """Мои ненаписанные стихи
в вихре дней, пробегающих мимо -
это тоже стихи, но стихи-
пантомима...""",
    """Все стихи с пометою “Вологда”
были созданы мной, между прочим,
в закутке у мусоропровода
между двенадцатью и пятью ночи...""",
    """Если наступит такая пора,
что будет всем хорошо -
я из поэтов уйду в доктора
или куда ишшо...""",
    """Я с каждым годом становлюсь мудрей,
неспешно продвигаясь раз за разом
от станции Какой-То-Там-Андрей
до станции Тот-Самый-Широглазов...""",
    """Там, где слышны кривотолки
на полублатном языке,
лучше быть умным в ермолке,
чем дураком в армяке!""",
    """Опять я вляпался в общественную жизнь
и трачу драгоценные минуты
на поиски одной бездарной рифмы
к поскуднейшему слову “юбиляр”.""",
    """А вам бы все считать мои грехи,
ругать меня свиньей неблагодарной.
Вот кончу пить - и напишу стихи.
А вы трезвы, но все равно бездарны...""",
    """На себя не лгу специально,
но открою вам секрет:
я опасен социально,
потому что я - поэт!""",
    """За каплю покаянья на лице
готов простить повешенных на стеньге.
Но то, что я живу в Череповце,
нельзя простить ни за какие деньги!""",
    """Память мы увековечили,
закалили свою сталь...
Больше нам дождливым вечером
делать нечего. А жаль...""",
    """…Мои всегдашние причуды
с трудом ложатся на твое
остервенелое мытье
послеобеденной посуды…""",
    """О, как патриотичны черепане!
О, как порою думают они:
«А это ведь Рубцов о ресторане
сказал: «О, «Русь», храни себя, храни!»""",
    """Увидев свет в зашторенном окне,
Она затормозила у развилки
Своих дорог и вновь пришла ко мне
В печали, налегке и без бутылки.""",
    """Я после слов поставил многоточье
И задремал за кухонным столом.
Но не сумел присниться этой ночью
Какой-нибудь блондинке за углом…""",
    """Размытый путь, кривые тополя…
Все точно так, как в том стихотворенье…
Мне кажется, что здесь моя земля.
Или опять обманывает зренье?""",
    """Пришел рассвет в малиновой рубахе,
Плеснул росы в проклюнувшийся груздь.
И разогнал мои ночные страхи,
И расплескал мою ночную грусть…""",
    """Пока грустит озерная трава
На фоне августовского заката,
Я забываю нужные слова
И на блокнот взираю виновато…""",
    """Меня терзают смутные сомненья,
Которые ничем не утолить:
Бог пригласил меня на день рожденья,
А вдруг приду – и могут не пустить…""",
    """Обрушив поэтическую муть
На чистый космос жизненной криницы,
В который раз стремятся обмануть
Меня вот эти книжные страницы…""",
    """Простите, я сегодня буду строг
(Вы не меня – вы Музу оскорбили!):
Когда стихи стучались в ваш мирок,
Вы им специально двери не открыли!""",
    """Зима!.. Крестьянин, торжествуя,
На дровнях обновляет путь.
Его лошадка, снег почуя,
Плетётся рысью как-нибудь.""",
    """Октябрь уж наступил — уж роща отряхает
Последние листы с нагих своих ветвей.
Дохнул осенний хлад — дорога промерзает,
Журча, ещё бежит за мельницу ручей.""",
    """Я вас любил, любовь ещё, быть может,
В душе моей угасла не совсем.
Но пусть она вас больше не тревожит,
Я не хочу печалить вас ничем.""",
    """Под голубыми небесами,
Великолепными коврами,
На солнце блещет снег искристый,
Лежат пушистые хребты.""",
    """Вот север, тучи нагоняя,
Дохнул, завыл — и вот сама
Идёт волшебница зима,
Снегами сыпля и сверкая.""",
    """Мой друг, забыты мной следы минувших лет
И младости моей мятежное теченье.
Не спрашивай меня о том, чего уж нет,
Оставь в душе одно лишь сладкое мученье.""",
    """На холмах Грузии лежит ночная мгла,
Шумит Арагва предо мною.
Мне грустно и легко, печаль моя светла,
Печаль моя полна тобою.""",
    """Город пышный, город бедный,
Дух неволи, стройный вид,
Свод небес зелёно-бледный,
Скука, холод и гранит.""",
    """Унылая пора! Очей очарованье!
Приятна мне твоя прощальная краса.
Люблю я пышное природы увяданье,
В багрец и золото одетые леса.""",
    """Брожу ли я вдоль улиц шумных,
Вхожу ль во многолюдный храм,
Сижу ль меж юношей безумных,
Я предаюсь моим мечтам.""",
    """В пустыне чахлой и скупой,
На почве, зноем раскалённой,
Анчар, как грозный часовой,
Стоит — один во всей вселенной.""",
    """Простишь ли мне ревнивые мечты,
Моей любви безумное волненье?
Ты мне верна, и для моей мечты
Ты заменяешь всё — и вдохновенье.""",
    """Поэт! Не дорожи любовию народной.
Восторженных похвал пройдёт минутный шум.
Услышишь суд глупца и смех толпы холодной,
Но ты останься твёрд, спокоен и угрюм.""",
    """Средь шумного бала, случайно,
В тревоге мирской суеты,
Тебя я увидел, но тайна
Твои покрывала черты.""",
    """И долго буду тем любезен я народу,
Что чувства добрые я лирой пробуждал,
Что в мой жестокий век восславил я Свободу
И милость к падшим призывал.""",
    """Я помню чудное мгновенье,
Передо мной явилась ты,
Как мимолётное виденье,
Как гений чистой красоты.""",
    """Погасло дневное светило,
На море синее вечерний пал туман.
Шуми, шуми, послушное ветрило,
Волнуйся подо мной, угрюмый океан.""",
    """Не пой, красавица, при мне
Ты песен Грузии печальной:
Напоминают мне оне
Другую жизнь и берег дальный.""",
    """В дубраву осень золотую
Надела пёстрый свой убор.
И ветер свищет на просторе,
Гоня прощальный хор листвы.""",
    """Люблю я строгие черты
Твоих задумчивых очей.
В них отблеск сумрачной мечты
И тихий ропот прежних дней.""",
    """Стуча, часы в ночной тиши
Ведут размеренный разговор.
Они твердят нам о душе,
О скоротечности веков.""",
    """Горит над мостовой фонарь,
Шуршат колёса по камням.
И ночь, как синий траурный вуаль,
Ложится на дома.""",
    """Твоя улыбка — луч денницы,
Что озаряет скорбный путь.
И в сердце смолкнувшей темницы
Готова любовью воскреснуть.""",
    """Искусство — вечный властелин,
Укротитель земных тревог.
В нём жизни бурной властный стих,
И мирозданья строгий слог.""",
    """На фабриках усталый гром
Не умолкает до зари.
Над чёрным, запылённым домом
Встаёт заря в немой хандре.""",
    """Когда твой голос слышу я,
Всё вкруг молчит, затая дух.
Лишь сердце, помнящее стужу,
Стучит навстречу в такт у двух.""",
    """Уносит времени река
Всё, чем душа была полна.
Лишь память, хрупкая рука,
Хранит осколки от окна.""",
    """По набережной в поздний час
Идёт задумчиво мечта.
И фонарей дрожащий глаз
Колеблет зыбкий свет листа.""",
    """Вопрос в анкете: был ли я судим?
Я каждый день и каждый миг судим.
За день вчерашний я судим сегодня.
А завтра ад настанет за сегодня.""",
    """Несу раздумий ношу на подъем,
Как мне достичь удастся перевала?
Ведь их, раздумий, больше с каждым днем,
А сил моих все меньше… вовсе мало.""",
    """Какой-то праздник, Господи, прости.
Забрел. Меня в передний угол пряча,
Бокал вина должны бы поднести
К моим губам, но… микрофон подносят.""",
    """Ах, Пушкин, Пушкин! Митинги вокруг,
Листовки, крики, ругань, вопли, споры…
Но ты со мною, мой любезный друг,
У нас еще орлы парят. И тишина. И горы.""",
    """Внушали нам: страна, в полете ты,
В пути и на «великом переломе».
Но оказалась на ремонте ты,
Как старый лифт в плохом и грязном доме.""",
    """Ведь не с Казбеком вровень я стою,
Ведь я не поднебесная вершина,
Откуда ж снег на голову мою,
Откуда ж эти белые седины?""",
    """На всех, на всех в бинокль смотрели мы,
Все видели в большом увеличенье.
Теперь бинокль перевернули мы…
А где же вещи в истинном значенье?""",
    """Уходит вождь, приходит новый вождь,
Законы, заседанья, словопренья…
Земле нужны крестьяне, солнце, дождь,
А не нужны бумажные решенья.""",
    """Я в жизни столько выслушал речей,
Зевоту подавляя для приличья,
А в это время где-то пел ручей,
И где-то раздавалось пенье птичье.""",
    """Спасибо любви, что стихами меня одарила,
Спасибо стихам, что меня научили любить.
И все мне казалось, как ангел, любовь белокрыла,
Меж ним и тобою мной песен протянута нить.""",
    """От альпинистов, от людей бывалых
Я знаю: спуск труднее, чем подъем.
Мне сорок лет, достиг я перевала,
Мой путь трудней и круче с каждым днем.""",
    """Поет у моего окошка ветер
Ту песню, что претит мне издавна;
А ту, что мне милей всего на свете,
Поет он у соседского окна.""",
    """С какой вершины я ни брошу камня,
Он вниз летит и пропадает там,
В какой низине песня б ни пришла мне,
Она летит к моим родным горам.""",
    """Смерть унесет и нас, и все, чем мы живем.
Избегнет лишь любовь ее прикосновенья.
Так ворон прочь летит, когда он видит дом,
Где в теплом очаге горят еще поленья.""",
    """И ты, кого все чествуют и славят,
Не забывайся, ибо, может статься,
По лестнице, тебя ведущей наверх,
Тебе же предстоит еще спускаться.""",
    """Распределение земных щедрот
Порой несправедливо в жизни нашей.
Я стал беззуб – суют мне мясо в рот,
Был при зубах я – пробавлялся кашей.""",
    """О смерти нету мысли у героя,
Поет о ней поэт, пока он жив.
А смерть с бессмертьем смотрят на обоих,
Для них свои ворота отворив.""",
    """Жизнь – мельница, и годы мелет жернов,
Не зная отдыха, не зная сна,
И крутится тем легче и проворней,
Чем больше он уже смолол зерна.""",
    """Не будь сумы на свете да тюрьмы,
Я стал бы оптимистом самым рьяным.
Когда бы все бессмертны были мы,
Я был бы атеистом самым рьяным.""",
    """Мы, путники, спокойны в час,
Когда собак не замечаем.
Когда они не видят нас,
Еще спокойней мы бываем.""",
    """Бывает очень тяжело, когда
Уменье есть, но силы нет для дела.
Бывает очень тяжело, когда
Ты неумел, хоть силе нет предела.""",
    """«Скажи, зачем с горами и лесами
Ты говоришь, о странный человек?»
«Полезны всем беседы с мудрецами,
Все повидавшими за долгий век!»""",
    """Из всех дорог я три дороги знаю,
С которых не сойдешь ни ты, ни я.
Поэзия – одна, и смерть – другая.
А третья? Та у каждого – своя.""",
    """Огонь, долги, болезни, враг,
Невзгоды, протекающие крыши, –
Преуменьшает лишь дурак
Значенье перечисленного выше.""",
    """«Кому несладко жить у нас, людей?»
«Кому не верят, тяжело тому!»
«Ну а кому гораздо тяжелей?»
«Тому, кто сам не верит никому!»""",
    """«Что бесполезней, чем незрелый плод,
Который раньше времени сорвали?»
«Упавший плод, что на земле гниет
Лишь потому, что вовремя не сняли».""",
    """Те, для кого поэзия – лишь дельце,
Порой певцов блистательных теснят.
Не так ли испокон веков пришельцы
Исконных обитателей теснят?""",
    """Я знаю: ты ругаешь не меня –
Во мне другого видишь и ругаешь.
Я знаю: ты и хвалишь не меня –
Во мне себя увидел ты и хвалишь.""",
    """«Кто счастливее, чем человек.
Что недуга отродясь не знал?»
«Только тот, кто зависти вовек
Ни к врагу, ни к другу не питал!»""",
    """«Птицы, что молчите вы с рассвета?»
«Дождь идет, мы слушаем его!»
«Почему молчите вы, поэты?»
«Дождь идет, мы слушаем его!»""",
    """Стихи уж точно не слова
С набором рифм и расстановкой точек,
Пускай  их целая глава,
Стихи лишь то, что сказано меж строчек.""",
    """Сегодня нищий, завтра я король,
Банкир или любовник страстный.
Надеюсь, что чужую  роль,
Играя, прожил не напрасно.""",
    """Слышу, ветер запутался в листьях,
Там  на ветке качнулась луна.
Разве может быть несколько истин,
Разве можно любовь не узнать?""",
    """Извозчик! В Болдино быстрей,
Гони, мой дорогой, коней-
Нельзя секунды нам терять-
Не будет Болдино опять.""",
    """Там где кончаются мечты
Предельно будьте осторожны.
Где шансов нет, из пустоты,
Старуха смерть Вам скорчит рожи.""",
    """Мы не жалеем времени,
А время наше нас,
Летит, не зная стремени,
Как с крыльями Пегас.""",
    """Забудем о плохом,
Там много обвинений,
Вся жизнь идет легко,
Коль не подвластна лени.""",
    """Может правда, а может, нет
Только истина странная штука
Для кого-то она   секрет,
А к кому-то приходит без стука.""",
    """Мы все такие, разные под небом
Где не похожи эти облака,
Но точно знаю, где б я не был,
Я с Вами рядом на земле пока.""",
    """Нас разделяет  расстоянье,
Соединяет нас любовь
И только мы с тобою знаем,
Придется, свидеться ли вновь.""",
    """Как только забываешь корни
Несет по миру ветерок,
А где пустить их,  с кем поспоришь,
Тот до безумия далек.""",
    """Писать короткие стихи
Как  позабыть о притяженье,
Казалось бы, слова легки,
Но сколько весит вдохновенье!""",
    """Друзей так много в дни побед,
А в дни невзгод ряды редеют,
Тогда-то ясен весь секрет,
Друзьями быть не все умеют.""",
    """Как пахнет костер
На руках твоих нежных.
Любовь иль простор?
Прощай безмятежность!""",
    """Мы любим мир через себя,
И лишь себе, не знаем цену,
Живя затем, чтоб только брать,
Меняем счастье на измену.""",
    """Все возвращается обратно,
Туда где любят и где ждут,
Бывает без любви приятно,
Но только лишь на пять минут.""",
    """Он скомкал много строк,
В своих твореньях,
Никто не понял, что
Писал их гений.""",
    """Солнцу не важно хорош или плох
Светит всем ярко как будто,
Будь то с колючками чертополох,
Или в  цветках  незабудка.""",
    """Мы выбираем, выбирают нас,
Как будто бы играем в карты,
А за окном цветет весна,
Не выбирая, без азарта.""",
    """Как  хорошо обманутыми быть,
Не думать,  не искать ответа,
Чтобы не знать, куда дороги нить
Несется завершать сюжеты.""",
    """Тебя б я мог бессмертной сделать,
Сказал  Шекспир, взглянув в окно,
Но дева оказалась телом,
И чистым на столе  листком.""",
    """Нельзя остановить мгновенье,
Не досказать, не до любить.
Жизнь лишь возможность, а не мненье,
Не цирк, и не игра на бис.""",
    """О чем еще мы не сказали?
Теперь разорван жизни круг
Мы не всегда сильнее стали,
Но в памяти ты жив, мой друг.""",
    """Когда умирают друзья,
Хоть время сегодня продажно,
Обратно  вернуть их нельзя
Деньгами, звонком иль бумажкой.""",
    """Сомненья гложут наши души,
Чтоб их сломать, они их сушат,
И мир становится другим
В нем нет друзей, одни враги.""",
    """Там, где недавно шли бои,
Цветет земля полами,
Но это место все ж болит,
Пока тревожит  память.""",
    """Если правят нувориши,
Не ищи в семье излишек,
Эти горе-господа,
Съели прибыль без труда.""",
    """Сверху крикнули им "фас",
"ЮКОС" быстренько угас.
Бей своих, чужих не будет-
Раз боятся - значит любят.""",
    """Как часто не знает правитель,
Что время не вечно течет,
Стареют законы и свита,
А смерть уж готовит расчет.""",
    """Мы будем вечно жить,
Лет семьдесят иль сорок,
Чтоб умереть в тиши
Незавершенных споров.""",
    """Напялю шутовской колпак,
Чтобы не слышать обвинений,
Что говорит царю дурак,
Подчас не скажет даже гений.""",
    """Так мало слов, чтоб достучаться,
Так много суеты порой.
Играть в супругов, домочадцев
Свою, не понимая роль.""",
    """Народ молчит,
Правительство не слышит.
И лишь в ночи
Попискивают  мыши.""",
    """Здесь выживает тот кто не боится волн
И знает их губительную силу,
Лишь он не расслабляясь держит челн,
Волне на встречу чтоб в нос лодки била.""",
    """Истину знает мудрец,
Глупый чудные  названья,
Чтоб удивить, наконец
Мир недоступностью знанья.""",
    """Как трудно удержать мгновенье,
С ним не поспоришь, не поймешь.
Летит, ему не нужно денег,
Чтоб показать как правду ложь.""",
    """Мне не исправить свой порок,
Я до сих пор пишу стихи,
Могу уменьшить число строк,
Чтоб больше выкинуть плохих.""",
    """Грустя о прожитых годах,
И о любви порой не зрелой,
Грусть нас роняет часто в страх,
Где путаются чувства с телом.""",
    """Мы покорим весь мир, но не себя,
И упиваясь до конца победой,
С собой не сможем ничего забрать,
В тот мир, в котором нет конца и следа.""",
    """Всего каприз и пару строчек,
Всего лишь взгляд и есть стихи,
Жизнь скорректирует, что почерк,
Что дуновение стихий.""",
    """Нет правды и добра, лишь равновесье,
Подвесило незримые весы,
Совсем случайно, зацепив за месяц,
Что освещает старые часы.""",
    """Когда ни - будь я стану молодым,
И сбудутся мои желанья,
Не испарятся как манящий дым,
Как сказка сна, которым ранен.""",
    """У богатства много плюсов,
Но и там бывает грустно,
Деньги все ж таки обман
Тянут вниз любой карман.""",
    """Мир безграничен как стихи,
Которые безумно ищут,
Перо, что будет не плохим,
Чтоб дать волненьям духа пищу.""",
    """О законах судите проще,
О стране по улыбкам людей,
Безнадежность выносит на площадь,
А надежда полна от дел.""",
    """Китайцы правы - не корми дракона,
Исчезнет он, все суета сует.
От ямы на дороге, до закона,
В котором правды и закона нет.""",
    """Пишу отнюдь не дуракам,
А тем, кто может думать,
Мои стихи бывают срам,
Но в них полно изюма.""",
    """Что хочет автор,неизвестно.
Он жаждет, что-то совершить,
Слова всего лишь сгусток теста,
Лишь смысл им позволяет жить.""",
    """Зачем поэзии стихи?
Чтоб примирить искусство с жизнью,
Как у костра времен - ухи
Хлебнуть и вспомнить вкус  отчизны.""",
    """Если  цель заменится сравненьем,
Мы получим грустный результат,
То, что важно только чье - то мненье,
А не воплощенный в жизнь талант.""",
    """Я пришел не  топтать
Этот мир безрассудно,
Лишь хочу показать,
Эту истину людям.""",
    """Коль душа запоет,
Так и дождик запляшет,
Тут и стих не соврет,
А все сделает краше.""",
    """Война всегда удел тиранов,
Они не могут мирно жить,
Чужое счастье для них рана,
Их счастье кровь во имя лжи.""",
    """Пиши лишь только для людей,
Они простят нам все ошибки,
А критик станет только злей,
Пусть зависть будет ему пыткой.""",
    """Всего лишь пару слов и ветер,
Несет осеннюю листву,
Он так строптив, что не заметил,
Как стих наполнил пустоту.""",
    """Начнешь писать -
Приходит вдохновенье,
К концу листа
И тени нет от лени...""",
    """А если честно без прикрас,
Пишу без устали для Вас,
Но кто бы знал как много строчек,
Я выбросил, оставив прочерк...""",
    """Березка сережки свои сберегла,
Чтоб с ветром рассветным, качаясь,
Смотрелась заманчиво даже где мгла.
Венчалась с осенней печалью.""",
    """Поэт напишет, то о чем молчим,
И станет неугодным на "мгновенье",
Как доктор, дав лекарство, что горчит,
Как совесть, что пришла из тени.""",
    """Осенний лист ютится у порога,
И тучи гонит ветер неспроста,
Прекрасна жизнь - её любовь и строгость,
Как линии, прожилки у листа.""",
    """У глупости идей немного
 - Отнять побить и растоптать.
Ей так трудна с умом дорога,
Ум хочет много, почему то дать.""",
    """Мы не знаем, что будет дальше.
Хотя видели сотни раз.
Лишь осталось стряхнуть от фальши
От сомнения жизни экстаз.....""",
    """О чем мечтаем мы - свобода
Она внезапно стала модной,
А мода ищет стиль,внимание,
Но лишь свобода - понимание.""",
    """Мы играем как дети,
Свободой своей,
Ища новые сети,
В которых вольней.""",
    """Не бойся потерять- не потеряешь,
Не бойся смерти точно не умрешь,
А бойся быть хорошим - ты у края,
Где тихо примостилась ложь.""",
    """Жизнь - дорога, конец известен,
Только знай ты попал на пир,
Где бывают красивые песни,
И где можно влюбиться в мир.""",
    """Чем больше критики к кому -то,
Тем больше хочется спросить,
Откуда факты?   Будет круто,
От тех кто любит ложь носить.""",
    """Успех не когда все есть,
И когда все в жизни можешь.
Успех от слова-УСПЕТЬ,
Схватить за гриву возможность.""",
    """У справедливости нет закона,
Потому, что в законе есть судьи,
Которые могут не вспомнить.
Что закон сочиняли люди…""",
    """Не сотвори себе кумира,
И будешь счастлив ты вовек
Лишь по тому ,что умер мирно,
Совсем не мирный человек.""",
    """Нет хороших войн,
Потому, что пули,
Не поняв кто свой
К нам летят вдоль улиц.""",
    """Я немного опьянён тобой.
Потому что, посмотрев в глаза,
В них увидел солнце над рекой,
Что легла как девичья коса.""",
    """Да все мы граждане "не той страны",
В которой так хотелось бы родиться,
Вот только путают всё карты сны,
А в них "не та страна" счастливой снится.""",
    """Не будет войн когда народ разумен,
Чтоб не держать за лидеров, господ.
Которые считаю только суммы.
Что им война с разбоем принесет.""",
    """Что делит мир?
Политики с нулями,
Что дали мы?
Толкнули их же камень.""",
    """Поэзия сродни любви,
В ней нет незаконченных строчек,
И смысла в ней может не быть,
Когда дописать её хочешь.""",
    """Мечты, мечты - всего лишь призрак,
А сколько силы нам дают.
Что поднимают вверх от низа,
И дарят счастье и уют.""",
    """Что есть добро на самом деле,
В чем суть его - глубокий смысл?
Быть может в том, чтобы успели....
Согреть замерших от зимы.""",
    """Наш лучший менеджер- убийца.
Наш худший из писак - поэт.
Что не запачкав чьи- то лица.
Снес всю систему в туалет.""",
    """У автора сложное бремя.
Творить, чтобы не был простой.
Бывает вгоняет нас в темень.
Но чаще он греет в зимой.""",
    """Все что болит, оно всегда не с нами,
Ведь боль подтачивает нас,
И разрушая превращает в камень
Души не тлеющий алмаз.""",
]
# СОХРАНЕНИЕ НОВОГО ДАТАСЕТА
print(f"Создаём файл с {len(poems)} стихотворениями...")

with open('pushkin_poems_final.txt', 'w', encoding='utf-8') as file:
    for i, poem in enumerate(poems, 1):
        file.write(poem)
        file.write('\n\n')

        if i % 10 == 0:
            print(f"  Записано {i}/{len(poems)} стихотворений")

print(f"✅ Файл создан: 'pushkin_poems_final.txt'")
print(f"   Всего стихотворений: {len(poems)}")
# ============================================
# ПРОВЕРКА ФАЙЛА И СТАТИСТИКА
# ============================================

print("\n=== ПРОВЕРКА СОЗДАННОГО ФАЙЛА ===")

# 1. Проверяем размер файла
import os
file_size = os.path.getsize('pushkin_poems_final.txt')
print(f"Размер файла: {file_size} байт ({file_size/1024:.2f} КБ)")

# 2. Читаем и показываем структуру
with open('pushkin_poems_final.txt', 'r', encoding='utf-8') as file:
    content = file.read()

    # Считаем количество стихотворений (по двойным переносам строки)
    poems_count = content.count('\n\n') + 1
    print(f"Количество стихотворений в файле: {poems_count}")

    # Считаем общее количество символов
    print(f"Общее количество символов: {len(content)}")

    # Показываем первые 300 символов для проверки
    print("\nПервые 300 символов файла:")
    print("-" * 50)
    print(content[:300] + "...")
    print("-" * 50)

# 3. Показываем информацию о каждом стихотворении
print("\n=== ИНФОРМАЦИЯ ПО СТИХОТВОРЕНИЯМ ===")
for i, poem in enumerate(poems, 1):
    lines = poem.count('\n') + 1
    chars = len(poem)
    print(f"Стихотворение {i:2d}: {lines:2d} строк, {chars:3d} символов")

Создаём файл с 203 стихотворениями...
  Записано 10/203 стихотворений
  Записано 20/203 стихотворений
  Записано 30/203 стихотворений
  Записано 40/203 стихотворений
  Записано 50/203 стихотворений
  Записано 60/203 стихотворений
  Записано 70/203 стихотворений
  Записано 80/203 стихотворений
  Записано 90/203 стихотворений
  Записано 100/203 стихотворений
  Записано 110/203 стихотворений
  Записано 120/203 стихотворений
  Записано 130/203 стихотворений
  Записано 140/203 стихотворений
  Записано 150/203 стихотворений
  Записано 160/203 стихотворений
  Записано 170/203 стихотворений
  Записано 180/203 стихотворений
  Записано 190/203 стихотворений
  Записано 200/203 стихотворений
✅ Файл создан: 'pushkin_poems_final.txt'
   Всего стихотворений: 203

=== ПРОВЕРКА СОЗДАННОГО ФАЙЛА ===
Размер файла: 42774 байт (41.77 КБ)
Количество стихотворений в файле: 204
Общее количество символов: 23993

Первые 300 символов файла:
--------------------------------------------------
Бог поглядел с тоскою

In [14]:
import torch
from transformers import AutoTokenizer
import json

# Загружаем токенизатор для большой модели
tokenizer = AutoTokenizer.from_pretrained(
    "ai-forever/rugpt3large_based_on_gpt2",
    trust_remote_code=True
)

# Критически важная настройка для GPT-2
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"✅ Токенизатор загружен. Pad token: {tokenizer.pad_token}")

# Чтение датасета
with open('pushkin_poems_final.txt', 'r', encoding='utf-8') as file:
    full_text = file.read()

# Разделение на стихотворения
poems = [poem.strip() for poem in full_text.split('\n\n') if poem.strip()]

# Параметры токенизации (оптимально для четверостиший)
MAX_LENGTH = 128

tokenized_data = {
    'input_ids': [],
    'attention_mask': [],
    'texts': []
}

for poem in poems:
    encoded = tokenizer(
        poem,
        max_length=MAX_LENGTH,
        padding='max_length',
        truncation=True,
        return_tensors='pt',
        add_special_tokens=True
    )

    tokenized_data['input_ids'].append(encoded['input_ids'][0].tolist())
    tokenized_data['attention_mask'].append(encoded['attention_mask'][0].tolist())
    tokenized_data['texts'].append(poem)

# Сохраняем токенизированные данные
with open('tokenized_poems_final.json', 'w', encoding='utf-8') as f:
    json.dump(tokenized_data, f, ensure_ascii=False, indent=2)

print(f"✅ Токенизация завершена. Примеров: {len(poems)}")
print(f"   Длина последовательности: {MAX_LENGTH} токенов")

✅ Токенизатор загружен. Pad token: <pad>
✅ Токенизация завершена. Примеров: 203
   Длина последовательности: 128 токенов


In [15]:
# ============================================
# ШАГ 3: ЗАГРУЗКА МОДЕЛИ И НАСТРОЙКА LORA (ИСПРАВЛЕННЫЙ)
# ============================================

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, TaskType, get_peft_model
import torch

print("Загружаем модель ruGPT3Large...")

# 1. Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained(
    "ai-forever/rugpt3large_based_on_gpt2",
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"✅ Токенизатор загружен")

# 2. Загружаем модель БЕЗ квантования (чтобы избежать ошибок)
model = AutoModelForCausalLM.from_pretrained(
    "ai-forever/rugpt3large_based_on_gpt2",
    torch_dtype=torch.float16,  # Используем половинную точность
    device_map="auto",  # Автоматическое распределение по GPU/CPU
    trust_remote_code=True
)

print("✅ Модель загружена")

# 3. Настраиваем LoRA (упрощенная конфигурация)
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["c_attn", "c_proj"],  # Минимальный набор для GPT-2
)

# 4. Применяем LoRA
model = get_peft_model(model, lora_config)

# 5. ОТКЛЮЧАЕМ gradient checkpointing (источник ошибки!)
model.gradient_checkpointing_disable()

print("✅ LoRA настроена, gradient checkpointing отключен")

# 6. Проверяем обучаемые параметры
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"📊 Обучаемых параметров: {trainable_params:,} ({trainable_params/total_params*100:.1f}%)")

print("\n" + "="*60)
print("✅ ШАГ 3 ЗАВЕРШЕН! Модель готова к обучению.")
print("="*60)

Загружаем модель ruGPT3Large...
✅ Токенизатор загружен
✅ Модель загружена


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


✅ LoRA настроена, gradient checkpointing отключен
📊 Обучаемых параметров: 3,244,032 (0.4%)

✅ ШАГ 3 ЗАВЕРШЕН! Модель готова к обучению.


In [16]:
# ============================================
# ШАГ 4: ОБУЧЕНИЕ МОДЕЛИ (ИСПРАВЛЕННЫЙ)
# ============================================

from torch.utils.data import Dataset, DataLoader, random_split
from transformers import DataCollatorForLanguageModeling
from torch.optim import AdamW
from tqdm import tqdm
import json

print("\n" + "="*60)
print("НАЧИНАЕМ ОБУЧЕНИЕ")
print("="*60)

# 1. Загрузка данных
with open('tokenized_poems_final.json', 'r', encoding='utf-8') as f:
    tokenized_data = json.load(f)

# 2. Простой Dataset
class SimplePoemDataset(Dataset):
    def __init__(self, tokenized_data):
        self.input_ids = tokenized_data['input_ids']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'labels': torch.tensor(self.input_ids[idx], dtype=torch.long)
        }

dataset = SimplePoemDataset(tokenized_data)

# 3. Разделение данных (90/10)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print(f"   Обучающая выборка: {len(train_dataset)} примеров")
print(f"   Валидационная выборка: {len(val_dataset)} примеров")

# 4. DataLoader (упрощенный)
BATCH_SIZE = 2  # Для MacBook с ограниченной памятью

train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

# 5. Оптимизатор
optimizer = AdamW(
    model.parameters(),
    lr=2e-4,  # Немного выше learning rate
    weight_decay=0.01,
)

# 6. Обучение
NUM_EPOCHS = 10  # Уменьшаем эпохи для стабильности
best_val_loss = float('inf')

print(f"\n⚙️  Параметры обучения:")
print(f"   Эпохи: {NUM_EPOCHS}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Learning rate: {optimizer.param_groups[0]['lr']}")
print("="*60)

# Переводим модель в режим обучения
model.train()

for epoch in range(NUM_EPOCHS):
    print(f"\n📈 Эпоха {epoch+1}/{NUM_EPOCHS}")

    total_loss = 0
    model.train()  # Важно: убеждаемся, что модель в режиме обучения

    # Прогресс-бар
    train_bar = tqdm(train_dataloader, desc="Обучение")

    for batch in train_bar:
        # Перенос данных на устройство модели
        input_ids = batch['input_ids'].to(model.device)
        labels = batch['labels'].to(model.device)

        # ПРОСТОЙ forward pass
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Обновление весов
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        # Обновление прогресс-бара
        train_bar.set_postfix({'loss': f"{loss.item():.4f}"})

    # Валидация
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc="Валидация"):
            input_ids = batch['input_ids'].to(model.device)
            labels = batch['labels'].to(model.device)

            outputs = model(input_ids=input_ids, labels=labels)
            val_loss += outputs.loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    avg_train_loss = total_loss / len(train_dataloader)

    print(f"   Train loss: {avg_train_loss:.4f}")
    print(f"   Val loss: {avg_val_loss:.4f}")

    # Сохранение лучшей модели
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss

        # Сохраняем модель
        model.save_pretrained("best_poetry_model")
        tokenizer.save_pretrained("best_poetry_model")

        print(f"   🏆 Сохраняем лучшую модель!")

    # Очистка памяти
    torch.cuda.empty_cache() if torch.cuda.is_available() else None

print("\n" + "="*60)
print("🎉 ОБУЧЕНИЕ ЗАВЕРШЕНО!")
print(f"Лучшая валидационная ошибка: {best_val_loss:.4f}")
print("="*60)

# 7. Сохраняем финальную модель
model.save_pretrained("final_poetry_model")
tokenizer.save_pretrained("final_poetry_model")
print("✅ Финальная модель сохранена в 'final_poetry_model'")


НАЧИНАЕМ ОБУЧЕНИЕ
   Обучающая выборка: 182 примеров
   Валидационная выборка: 21 примеров

⚙️  Параметры обучения:
   Эпохи: 10
   Batch size: 2
   Learning rate: 0.0002

📈 Эпоха 1/10


Валидация: 100%|██████████| 11/11 [00:00<00:00, 19.17it/s]


   Train loss: 3.7961
   Val loss: 1.0873
   🏆 Сохраняем лучшую модель!

📈 Эпоха 2/10


Валидация: 100%|██████████| 11/11 [00:00<00:00, 25.12it/s]


   Train loss: 1.0720
   Val loss: 1.0252
   🏆 Сохраняем лучшую модель!

📈 Эпоха 3/10


...

   Train loss: 0.9343
   Val loss: 1.0137

📈 Эпоха 9/10


Валидация: 100%|██████████| 11/11 [00:00<00:00, 24.77it/s]


   Train loss: 0.9079
   Val loss: 1.0115

📈 Эпоха 10/10


Валидация: 100%|██████████| 11/11 [00:00<00:00, 25.31it/s]


   Train loss: 0.8817
   Val loss: 1.0254

🎉 ОБУЧЕНИЕ ЗАВЕРШЕНО!
Лучшая валидационная ошибка: 0.9973
✅ Финальная модель сохранена в 'final_poetry_model'


In [17]:
# ============================================
# ФИНАЛЬНЫЙ ГЕНЕРАТОР СТИХОВ (ПРОСТОЙ И РАБОЧИЙ)
# ============================================

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import os

print("🎭 ГЕНЕРАТОР СТИХОВ")
print("=" * 50)

# 1. ПРОВЕРКА МОДЕЛИ
if not os.path.exists("final_poetry_model"):
    print("❌ Модель не найдена! Сначала обучите модель.")
    exit()

print("📂 Загружаем модель...")

# 2. ЗАГРУЗКА
base_model = AutoModelForCausalLM.from_pretrained(
    "ai-forever/rugpt3large_based_on_gpt2",
    device_map="auto",
    torch_dtype=torch.float16
)

model = PeftModel.from_pretrained(base_model, "final_poetry_model")
tokenizer = AutoTokenizer.from_pretrained("final_poetry_model")

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model.eval()
print("✅ Модель загружена")

# 3. ПРОСТАЯ ФУНКЦИЯ ГЕНЕРАЦИИ С ОПТИМАЛЬНЫМИ ПАРАМЕТРАМИ
def generate_good_poem(prompt, lines=4):
    """Генерирует качественные стихи"""

    # Делаем промпт лучше
    if len(prompt.split()) < 3:
        clean_prompt = f"Стих о {prompt}"
    else:
        clean_prompt = prompt

    # Токенизация
    inputs = tokenizer(clean_prompt, return_tensors="pt").to(model.device)

    # ОПТИМАЛЬНЫЕ ПАРАМЕТРЫ для стихов:
    # temperature=0.72 - идеальный баланс для рифмы
    # top_p=0.88 - ограничивает выбор слов
    # repetition_penalty=1.18 - предотвращает повторы
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=lines * 25,  # ~25 токенов на строку
            temperature=0.68,           # Лучшая температура для стихов
            top_p=0.88,                 # Хороший контроль качества
            do_sample=True,             # Обязательно для temperature
            repetition_penalty=1.18,    # Убирает повторения
            no_repeat_ngram_size=3,     # Не повторять фразы
            pad_token_id=tokenizer.pad_token_id,
        )

    # Получаем результат
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Убираем промпт
    if result.startswith(clean_prompt):
        result = result[len(clean_prompt):]

    # Простая очистка
    poem_lines = []
    for line in result.split('\n'):
        line = line.strip()
        if line and 8 <= len(line) <= 60:  # Нормальная длина строки
            poem_lines.append(line)

    # Возвращаем нужное количество строк
    if len(poem_lines) > lines:
        poem_lines = poem_lines[:lines]

    return '\n'.join(poem_lines) if poem_lines else "Не удалось сгенерировать"

# 4. ТЕСТ РАБОТЫ
print("\n🧪 ТЕСТИРУЕМ С temperature=0.72")
print("=" * 40)

test_words = ["Мороз и солнце", "У лукоморья дуб", "Осенний дождь"]

for word in test_words:
    print(f"\n📝 '{word}':")
    print("-" * 30)
    poem = generate_good_poem(word, lines=4)
    print(poem)

# 5. ПРОСТОЙ ИНТЕРАКТИВНЫЙ РЕЖИМ
print("\n" + "=" * 50)
print("✨ ИНТЕРАКТИВНЫЙ РЕЖИМ")
print("=" * 50)
print("Вводите тему для стиха.")
print("Например: зима, любовь, море, ночной город")
print("Для выхода введите 'выход' или оставьте пусто")
print("=" * 50)

count = 0

while True:
    user_input = input(f"\n{count+1}> Тема: ").strip()

    if user_input.lower() in ['выход', 'exit', '']:
        print("👋 Завершение...")
        break

    print("⚙️  Генерация...")

    # Автоматически решаем, сколько строк
    if len(user_input.split()) > 2:
        num_lines = 6  # Для длинных тем - больше строк
    else:
        num_lines = 4  # Для коротких - четверостишие

    try:
        poem = generate_good_poem(user_input, lines=num_lines)
        count += 1

        print(f"\n📜 Стих №{count} ({num_lines} строк):")
        print("-" * 40)
        print(poem)
        print("-" * 40)

        # Сохраняем в файл
        with open("мои_стихи.txt", "a", encoding="utf-8") as f:
            f.write(f"\n{'-'*40}\n")
            f.write(f"Тема: {user_input}\n")
            f.write(f"{poem}\n")

        print(f"💾 Сохранено в 'мои_стихи.txt'")
    except Exception as e:
        print(f"❌ Ошибка: {e}")
        print("🔄 Пробую ещё раз...")

        # Пробуем с более простыми параметрами
        try:
            poem = generate_good_poem(user_input, lines=4)
            print(f"\n📜 Результат:\n{poem}")
        except:
            print("😞 Не получилось. Попробуйте другую тему.")

# 6. ИТОГИ
print("\n" + "=" * 50)
print("📊 ИТОГ:")
print(f"Сгенерировано стихов: {count}")
print(f"Все стихи в файле: 'мои_стихи.txt'")
print("=" * 50)
print("✅ Готово! Параметры: temperature=0.72, top_p=0.88")
print("   Это оптимальные настройки для качественных стихов.")
print("=" * 50)

🎭 ГЕНЕРАТОР СТИХОВ
📂 Загружаем модель...
✅ Модель загружена

🧪 ТЕСТИРУЕМ С temperature=0.72

📝 'Мороз и солнце':
------------------------------
, день чудесный!
Хочу сказать я вам от души:
Пусть будет в жизни много смеха,
И пусть не страшны будут ненастья.

📝 'У лукоморья дуб':
------------------------------
зеленый,
Там чудеса: там леший бродит…
А у нас - колбаса с сыром.

📝 'Осенний дождь':
------------------------------
Евгений Лебедев
Текст предоставлен издательством «Эксмо»
© Е. Лебедев, 2017
Создано в интеллектуальной издательской системе Ridero

✨ ИНТЕРАКТИВНЫЙ РЕЖИМ
Вводите тему для стиха.
Например: зима, любовь, море, ночной город
Для выхода введите 'выход' или оставьте пусто

1> Тема: напиши стихотворение в 4 строки на тему лето
⚙️  Генерация...

📜 Стих №1 (6 строк):
----------------------------------------
Лети, птица! лети! Над рекою дымкой вьётся,
Над лесами, полями и болотами,
Где так много разных птиц живёт...
Я знаю - ты прилетаешь к нам летом,
В страну чудесных песен и

KeyboardInterrupt: Interrupted by user